In [1]:
import dagshub
dagshub.init(repo_owner='leocus4', repo_name='TinyFFF', mlflow=True)

Repository initialized!

In [2]:
# IMPORT FOR COMPRESSION
import sys
sys.path.insert(0, './Compression-Fast-Inf-FFF/')
from sparsecompFFF import compress_FF_models, print_size_model
from save_torch_model_to_c import main

In [14]:
import torch
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
device = DEVICE
print(f"Training on {DEVICE}")

Training on cuda


In [4]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

In [5]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

In [6]:
from fastfeedforward import FFF

def train(net, trainloader, epochs, norm_weight=0.0):
    """Train the network on the training set."""
    # Define loss and optimizer
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

    # Train the network for the given number of epochs
    for _ in range(epochs):
        # Iterate over data
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            loss = criterion(net(images), labels)
            if norm_weight != 0:
                loss += norm_weight * net.fff.w1s.pow(2).sum()
                loss += norm_weight * net.fff.w2s.pow(2).sum()
            loss.backward()
            optimizer.step()


def test(net, testloader):
    """Validate the network on the entire test set."""
    # Define loss and optimizer
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    # Train the network for the given number of epochs
    with torch.no_grad():
        # Iterate over data
        for data in testloader:
            images, labels = data[0].to(DEVICE), data[1].to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return loss, accuracy


class Net(torch.nn.Module):
    def __init__(self, input_width, leaf_width, output_width, depth, dropout, region_leak):
        super(Net, self).__init__()
        self.fff = FFF(input_width, leaf_width, output_width, depth, torch.nn.ReLU(), dropout, train_hardened=True, region_leak=region_leak)

    def forward(self, x):
        x = x.view(len(x), -1)
        x = self.fff(x)
        x = torch.nn.functional.softmax(x, -1)
        return x

    def parameters(self):
        return self.fff.parameters()


class FF(torch.nn.Module):
    def __init__(self, input_width, layer_width, output_width):
        super(FF, self).__init__()
        self.fc1 = torch.nn.Linear(input_width, layer_width)
        self.fc2 = torch.nn.Linear(layer_width, output_width)

    def forward(self, x):
        x = x.view(len(x), -1)
        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.softmax(self.fc2(x), -1)
        return x

    def parameters(self):
        return [*self.fc1.parameters(), *self.fc2.parameters()]


def compute_n_params(input_width: int, l_w: int, depth: int, output_width: int):
    fff = Net(input_width, l_w, output_width, depth, 0, 0)
    ff = FF(input_width, l_w, output_width)

    n_ff = 0
    n_fff = 0
    for p in ff.parameters():
        n_ff += p.numel()
    for i, p in enumerate(fff.parameters()):
        print(f"[{i}-th layer]: {p.shape}")
        n_fff += p.numel()

    print(f"FFF: {n_fff}\nFF: {n_ff}")

In [7]:
import pickle
import mlflow
import numpy as np
import pandas as pd
from time import time
from matplotlib import pyplot as plt

def get_dist(net, testloader):
    """
    Returns the distribution of samples throughout the tree.
    """

    y = []
    l = []
    with torch.no_grad():
        # Iterate over data
        for data in testloader:
            images, labels = data[0].to(DEVICE), data[1].to(DEVICE)
            outputs, leaves = net.forward(images, return_nodes=True)
            y.append(labels)
            l.append(leaves)
    y = torch.concat(y, 0)
    l = torch.concat(l, 0)
    return y, l


class FFFWrapper(torch.nn.Module):
    def __init__(self, fff):
        super(FFFWrapper, self).__init__()
        self._fff = fff
        self._fastinference = [None for i in range(2 ** (self._fff.fff.depth.item()))]

    def forward(self, x, return_nodes=False):
        """
        Override the forward method in order to log the data distribution.
        """
        x = x.view(len(x), -1)
        original_shape = x.shape
        batch_size = x.shape[0]
        last_node = torch.zeros(len(x))

        current_nodes = torch.zeros((batch_size,), dtype=torch.long, device=x.device)
        for i in range(self._fff.fff.depth.item()):
            plane_coeffs = self._fff.fff.node_weights.index_select(dim=0, index=current_nodes)
            plane_offsets = self._fff.fff.node_biases.index_select(dim=0, index=current_nodes)
            plane_coeff_score = torch.bmm(x.unsqueeze(1), plane_coeffs.unsqueeze(-1))
            plane_score = plane_coeff_score.squeeze(-1) + plane_offsets
            plane_choices = (plane_score.squeeze(-1) >= 0).long()

            platform = torch.tensor(2 ** i - 1, dtype=torch.long, device=x.device)
            next_platform = torch.tensor(2 ** (i+1) - 1, dtype=torch.long, device=x.device)
            current_nodes = (current_nodes - platform) * 2 + plane_choices + next_platform

        leaves = current_nodes - next_platform
        new_logits = torch.empty((batch_size, self._fff.fff.output_width), dtype=torch.float, device=x.device)
        last_node = leaves

        for i in range(leaves.shape[0]):
            leaf_index = leaves[i]
            if self._fastinference[leaf_index] is not None:
                new_logits[i] = self._fastinference[leaf_index]
            else:
                logits = torch.matmul( x[i].unsqueeze(0), self._fff.fff.w1s[leaf_index])
                logits += self._fff.fff.b1s[leaf_index].unsqueeze(-2)
                activations = self._fff.fff.activation(logits)
                new_logits[i] = torch.matmul( activations, self._fff.fff.w2s[leaf_index]).squeeze(-2)

        if return_nodes:
            return new_logits.view(*original_shape[:-1], self._fff.fff.output_width), last_node
        return new_logits.view(*original_shape[:-1], self._fff.fff.output_width)


    def simplify_leaves(self, trainloader):
        y, leaves = (get_dist(self, trainloader))
        y = y.cpu().detach().numpy()
        outputs = y.max() + 1
        leaves = leaves.cpu().detach().numpy()

        n_simplifications = 0
        ratios = {}
        for l in np.unique(leaves):
            ratios[l] = torch.zeros(outputs)
            indices = leaves == l

            for i in range(outputs):
                ratios[l][i] = (np.sum(y[indices] == i) / np.sum(indices))

            argmax = np.argmax(ratios[l])
            if ratios[l][argmax] > 0.7:
                output = torch.zeros(outputs)
                output[argmax] = 1
                self._fastinference[l] = output
                n_simplifications += 1
                print(f"Leaf {l} has been replaced with {argmax}")
        print(self._fastinference)

In [8]:
class FFFWrapper(torch.nn.Module):
    def __init__(self, fff):
        super(FFFWrapper, self).__init__()
        self._fff = fff
        self._fastinference = [None for i in range(2 ** (self._fff.fff.depth.item()))]

    def forward(self, x, return_nodes=False):
        """
        Override the forward method in order to log the data distribution.
        """
        x = x.view(len(x), -1)
        original_shape = x.shape
        batch_size = x.shape[0]
        last_node = torch.zeros(len(x))

        current_nodes = torch.zeros((batch_size,), dtype=torch.long, device=x.device)
        for i in range(self._fff.fff.depth.item()):
            plane_coeffs = self._fff.fff.node_weights.index_select(dim=0, index=current_nodes)
            plane_offsets = self._fff.fff.node_biases.index_select(dim=0, index=current_nodes)
            plane_coeff_score = torch.bmm(x.unsqueeze(1), plane_coeffs.unsqueeze(-1))
            plane_score = plane_coeff_score.squeeze(-1) + plane_offsets
            plane_choices = (plane_score.squeeze(-1) >= 0).long()

            platform = torch.tensor(2 ** i - 1, dtype=torch.long, device=x.device)
            next_platform = torch.tensor(2 ** (i+1) - 1, dtype=torch.long, device=x.device)
            current_nodes = (current_nodes - platform) * 2 + plane_choices + next_platform

        leaves = current_nodes - next_platform
        new_logits = torch.empty((batch_size, self._fff.fff.output_width), dtype=torch.float, device=x.device)
        last_node = leaves

        for i in range(leaves.shape[0]):
            leaf_index = leaves[i]
            if self._fastinference[leaf_index] is not None:
                new_logits[i] = self._fastinference[leaf_index]
            else:
                logits = torch.matmul( x[i].unsqueeze(0), self._fff.fff.w1s[leaf_index])
                logits += self._fff.fff.b1s[leaf_index].unsqueeze(-2)
                activations = self._fff.fff.activation(logits)
                new_logits[i] = torch.matmul( activations, self._fff.fff.w2s[leaf_index]).squeeze(-2)

        if return_nodes:
            return new_logits.view(*original_shape[:-1], self._fff.fff.output_width), last_node
        return new_logits.view(*original_shape[:-1], self._fff.fff.output_width)


    def simplify_leaves(self, trainloader):
        y, leaves = (get_dist(self, trainloader))
        y = y.cpu().detach().numpy()
        outputs = y.max() + 1
        leaves = leaves.cpu().detach().numpy()

        n_simplifications = 0
        ratios = {}
        for l in np.unique(leaves):
            ratios[l] = torch.zeros(outputs)
            indices = leaves == l

            for i in range(outputs):
                ratios[l][i] = (np.sum(y[indices] == i) / np.sum(indices))

            argmax = np.argmax(ratios[l])
            if ratios[l][argmax] > 0.7:
                output = torch.zeros(outputs)
                output[argmax] = 1
                self._fastinference[l] = output
                n_simplifications += 1
                print(f"Leaf {l} has been replaced with {argmax}")
        print(self._fastinference)
    
    def simplify_leaves_full(self, trainloader):
        y, leaves = (get_dist(self, trainloader))
        y = y.cpu().detach().numpy()
        outputs = y.max() + 1
        leaves = leaves.cpu().detach().numpy()

        n_simplifications = 0
        ratios = {}
        for l in np.unique(leaves):
            ratios[l] = torch.zeros(outputs)
            indices = leaves == l

            for i in range(outputs):
                ratios[l][i] = (np.sum(y[indices] == i) / np.sum(indices))

            argmax = np.argmax(ratios[l])
            if ratios[l][argmax] >= 0:
                output = torch.zeros(outputs)
                output[argmax] = 1
                self._fastinference[l] = output
                n_simplifications += 1
                print(f"Leaf {l} has been replaced with {argmax}")
        print(self._fastinference)

In [9]:
from torch.utils.data import DataLoader, Dataset
class SpeechDataset(Dataset):
    def __init__(self, fold="train"):
        df = pd.read_csv(f"../MobiCom2024/features/sa_{fold}.csv")
        self.data = []
        self.labels = []
        for _, (dir, name, label) in df.iterrows():
            self.data.append(np.load(f"../MobiCom2024/features/{dir}/{name.replace('wav', 'npy')}"))
            self.labels.append(label)
        self.data = np.array(self.data).astype(np.float32)
        self.labels = np.array(self.labels).astype(np.int64)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data[index]
        y = self.labels[index]
        return x, y

In [10]:
trainset = SpeechDataset("train")
testset = SpeechDataset("test")
valset = SpeechDataset("val")

In [11]:
from tqdm import trange
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

# Select class to keep 
train_loader = DataLoader(trainset, batch_size=512, shuffle=True)
val_loader = DataLoader(valset, batch_size=512)
test_loader = DataLoader(testset, batch_size=512)

num_examples = {"trainset" : len(trainset), "valset" : len(valset), "testset" : len(testset)}

print(num_examples)

{'trainset': 26982, 'valset': 3854, 'testset': 7710}


In [12]:
list_of_run = [
'9cc6a99bbdaf47ea93157add2e3abff5',
]

# mlflow.artifacts.download_artifacts(run_id=run_id, dst_path=".")
# wrapped_model = pickle.load(open("./truncated_model.pkl", "rb"))
# wrapped_model._fff.fff.depth.item()
# wrapped_model._fff.fff.input_width
# wrapped_model._fff.fff.leaf_width
# wrapped_model._fff.fff.output_width

In [15]:
result = []
for i in range (0,len(list_of_run)):
    run_id = list_of_run[i]
    
    run = mlflow.get_run(run_id)
    starting_run_id = run.data.params['starting_run']
    starting_run = mlflow.get_run(starting_run_id)
    norm_weight = starting_run.data.params['norm_weight']
    #print(norm_weight)
    
    mlflow.artifacts.download_artifacts(run_id=run_id, dst_path="./baselines")
    wrapped_model = pickle.load(open("./baselines/truncated_model.pkl", "rb"))
    depth = wrapped_model._fff.fff.depth.item()
    input_width = wrapped_model._fff.fff.input_width
    leaf_width = wrapped_model._fff.fff.leaf_width
    output_width = wrapped_model._fff.fff.output_width
    buffer_size = 2*(leaf_width + output_width + 3)
    print("Run:\t", run_id)
    print("Depth:\t", depth)
    print("Input:\t", input_width)
    print("Output:\t", output_width)
    print("Leaf:\t", leaf_width)
    print("Buffer:\t", buffer_size)
    
    # to reduce the sparsity and train only below a certain tresholds
    list_of_sizes = [100, 90, 80, 70, 60, 50]
    checked_sizes = [False for x in list_of_sizes]
    current_size_index = 0
    
    start = 0.5
    a = start
    b = start
    sizes=[]
    before_trunc_sizes=[]
    trunc_sizes=[]

    model = wrapped_model.to(device)
    
    layers_list = []
    for i, (name, p) in enumerate(model.named_parameters()):
        if (len(list(p.shape)) > 1 and p.requires_grad):
            layers_list.append(p)
    un, comp, layers = print_size_model(model, layers_list, [1,1,1,1,1,1])
    
    result.append({'run_id': run_id, 'depth': depth,'input_width':input_width, 
                   'output': output_width, 'leaf_width':leaf_width,
                   'buffer_size': buffer_size, 'sizes': un})

Run:	 9cc6a99bbdaf47ea93157add2e3abff5
Depth:	 3
Input:	 793
Output:	 10
Leaf:	 32
Buffer:	 90


In [16]:
for r in result:
    print(r)

{'run_id': '9cc6a99bbdaf47ea93157add2e3abff5', 'depth': 3, 'input_width': 793, 'output': 10, 'leaf_width': 32, 'buffer_size': 90, 'sizes': 845856}


In [17]:
compress_FF_models(model, 50, train_loader, test_loader, val_loader=val_loader, num_epochs=8)

Target size requested: 50 KB
Starting Density of model's parameters: [1, 1, 1, 1, 1, 1]
Starting size of the model: 845.856 KB
1 iteration -  Size: 764684.8 [1, 1, 0.45, 1, 1, 1]
2 iteration -  Size: 691601.92 [1, 1, 0.405, 1, 1, 1]
3 iteration -  Size: 625827.3280000001 [1, 1, 0.36450000000000005, 1, 1, 1]
4 iteration -  Size: 566630.1952000001 [1, 1, 0.32805000000000006, 1, 1, 1]
5 iteration -  Size: 513352.7756800001 [1, 1, 0.29524500000000004, 1, 1, 1]
6 iteration -  Size: 465403.09811200004 [1, 1, 0.2657205, 1, 1, 1]
7 iteration -  Size: 422248.3883008 [1, 1, 0.23914845, 1, 1, 1]
8 iteration -  Size: 383409.14947072 [1, 1, 0.215233605, 1, 1, 1]
9 iteration -  Size: 348453.83452364797 [1, 1, 0.1937102445, 1, 1, 1]
10 iteration -  Size: 316994.0510712832 [1, 1, 0.17433922005, 1, 1, 1]
11 iteration -  Size: 288680.2459641549 [1, 1, 0.156905298045, 1, 1, 1]
12 iteration -  Size: 263197.82136773935 [1, 1, 0.1412147682405, 1, 1, 1]
13 iteration -  Size: 240263.63923096546 [1, 1, 0.12709

48673.96288481359

In [18]:
compress_FF_models(model, 40, train_loader, test_loader, val_loader=val_loader, fastInfLoss=True, fastInfNormWeight=0.0001, num_epochs=8, device=device)

Target size requested: 40 KB
Starting Density of model's parameters: [1, 1, 0.00912771910466581, 1, 1, 1]
Starting size of the model: 48.64799999999997 KB
1 iteration -  Size: 49635.59999999998 [0.45, 1, 0.00912771910466581, 1, 1, 1]
2 iteration -  Size: 47637.239999999976 [0.405, 1, 0.00912771910466581, 1, 1, 1]
3 iteration -  Size: 45838.71599999997 [0.36450000000000005, 1, 0.00912771910466581, 1, 1, 1]
4 iteration -  Size: 44220.04439999998 [0.32805000000000006, 1, 0.00912771910466581, 1, 1, 1]
5 iteration -  Size: 42763.23995999998 [0.29524500000000004, 1, 0.00912771910466581, 1, 1, 1]
6 iteration -  Size: 41452.115963999975 [0.2657205, 1, 0.00912771910466581, 1, 1, 1]
7 iteration -  Size: 40272.104367599975 [0.23914845, 1, 0.00912771910466581, 1, 1, 1]
Disabling: _fff.fff.depth
Disabling: _fff.fff.node_weights
Disabling: _fff.fff.node_biases
Disabling: _fff.fff.w1s
Disabling: _fff.fff.b1s
Disabling: _fff.fff.w2s
Disabling: _fff.fff.b2s
Activating: torch.Size([7, 793])
Activating: 

39210.093930839976

In [19]:
original_fastinference = str([-1 if x is None else int(x.argmax()) for x in model._fastinference])
main(model, "model_to_c", original_fastinference)